# Experiment Comparison

This notebook demonstrates how to query and compare MLflow experiment runs
from the XJTU-SY bearing RUL training pipeline.

**Prerequisites**: Run at least one training fold via
`python scripts/05_train_dl_models.py --model <model_name> --folds 0`
so that MLflow runs exist in `mlruns/`.

In [ ]:
import sys
sys.path.insert(0, "..")

import mlflow
import pandas as pd

# Point to local MLflow tracking store
mlflow.set_tracking_uri("../mlruns")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

In [ ]:
# Cell 2: List all experiments
experiments = mlflow.search_experiments()
print(f"Found {len(experiments)} experiment(s):\n")
for exp in experiments:
    print(f"  ID: {exp.experiment_id}  Name: {exp.name}  Artifact location: {exp.artifact_location}")

In [ ]:
# Cell 3: List all runs for the bearing_rul_dl experiment
EXPERIMENT_NAME = "bearing_rul_dl"

runs_df = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    order_by=["start_time DESC"],
)

if runs_df.empty:
    print(f"No runs found for experiment '{EXPERIMENT_NAME}'.")
    print("Run training first: python scripts/05_train_dl_models.py --model cnn1d_baseline --folds 0")
else:
    print(f"Found {len(runs_df)} run(s):\n")
    # Show key columns
    display_cols = ["run_id", "status", "start_time"]
    param_cols = [c for c in runs_df.columns if c.startswith("params.")]
    metric_cols = [c for c in runs_df.columns if c.startswith("metrics.")]
    display_cols += param_cols[:6] + metric_cols[:6]
    display(runs_df[[c for c in display_cols if c in runs_df.columns]])

In [ ]:
# Cell 4: Compare runs across folds using the project's compare_runs() utility
from src.utils.tracking import ExperimentTracker, compare_runs

tracker = ExperimentTracker(
    backend="mlflow",
    experiment_name=EXPERIMENT_NAME,
    tracking_uri="../mlruns",
)

runs = tracker.list_runs(experiment_name=EXPERIMENT_NAME)
print(f"Retrieved {len(runs)} RunInfo objects")

if runs:
    comparison = compare_runs(
        runs,
        metrics=["final_rmse", "final_mae", "final_phm08_score"],
        params=["model_name", "fold_id"],
    )
    print("\n--- Metric Statistics ---")
    for metric_name, stats in comparison["metric_stats"].items():
        print(f"  {metric_name}: mean={stats['mean']:.2f}, std={stats['std']:.2f}, "
              f"min={stats['min']:.2f}, max={stats['max']:.2f}")

    if comparison["best_run"]:
        best = comparison["best_run"]
        print(f"\nBest run: {best.run_name} (ID: {best.run_id[:8]}...)")

    # Show summary table
    summary_df = comparison["dataframe"]
    display(summary_df)
else:
    print("No runs available for comparison.")

In [ ]:
# Cell 5: Plot metric comparison bar chart using the project's plot_metric_comparison()
from src.utils.tracking import plot_metric_comparison
import matplotlib.pyplot as plt

if runs:
    fig = plot_metric_comparison(runs, metric_name="final_rmse", figsize=(12, 5))
    plt.show()

    fig = plot_metric_comparison(runs, metric_name="final_mae", figsize=(12, 5))
    plt.show()
else:
    print("No runs to plot. Train a model first.")

In [ ]:
# Cell 6: Export comparison report to HTML
from src.utils.tracking import export_comparison_report
from pathlib import Path

if runs:
    report_path = Path("../outputs/evaluation/experiment_comparison_report.html")
    export_comparison_report(runs, output_path=report_path, format="html")
    print(f"Report saved to: {report_path.resolve()}")
    print(f"Open in browser: file://{report_path.resolve()}")
else:
    print("No runs to export. Train a model first.")